In [2]:
!pip install pyspark


In [2]:
pwd

'/content'

In [5]:
!ls


sample_data


In [6]:
ls -l

total 4
drwxr-xr-x 1 root root 4096 Sep 12 13:22 sample_data/


In [7]:
!man ls

This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, including manpages, you can run the 'unminimize'
command. You will still need to ensure the 'man-db' package is installed.


In [3]:
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv


--2024-09-25 14:32:01--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv [following]
--2024-09-25 14:32:01--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22608 (22K) [text/csv]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  22.08K  --.-KB/s    in 0.001s  

2024-09-25 14:32:01 (23.3 MB/s) - ‘cars.csv’ saved [22608/22608]



In [10]:
!ls -l

total 28
-rw-r--r-- 1 root root 22608 Aug  1 13:43 cars.csv
drwxr-xr-x 1 root root  4096 Sep 12 13:22 sample_data


In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cars.csv').getOrCreate()

df = spark.read.csv('cars.csv', header=True, sep=";", inferSchema=True)
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0|  4354|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|

In [5]:
from pyspark.sql.functions import when

df = df.withColumn("Warranty", when(df.Origin == "US", "10 Years").otherwise("5 Years"))
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Warranty|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|10 Years|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|10 Years|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|10 Years|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|10 Years|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|10 Years|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|10 Years|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0|  4

In [9]:
df.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: decimal(4,0) (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Warranty: string (nullable = false)



In [10]:
renamed_df = df.withColumnRenamed("Car", "Car_Model") \
               .withColumnRenamed("MPG", "Miles_per_Gallon") \
               .withColumnRenamed("Cylinders", "No_of_Cylinders") \
               .withColumnRenamed("Displacement", "Displacement_No")\
               .withColumnRenamed("Horsepower", "Horsepower_No") \
               .withColumnRenamed("Weight", "Weight_No") \
               .withColumnRenamed("Acceleration", "Acceleration_No") \
               .withColumnRenamed("Model", "Year_of_Manufacture") \
               .withColumnRenamed("Origin", "Country_of_Origin") \
               .withColumnRenamed("Warranty", "Warranty_Years")
renamed_df.show()

+--------------------+----------------+---------------+---------------+-------------+---------+---------------+-------------------+-----------------+--------------+
|           Car_Model|Miles_per_Gallon|No_of_Cylinders|Displacement_No|Horsepower_No|Weight_No|Acceleration_No|Year_of_Manufacture|Country_of_Origin|Warranty_Years|
+--------------------+----------------+---------------+---------------+-------------+---------+---------------+-------------------+-----------------+--------------+
|Chevrolet Chevell...|            18.0|              8|          307.0|        130.0|     3504|           12.0|                 70|               US|      10 Years|
|   Buick Skylark 320|            15.0|              8|          350.0|        165.0|     3693|           11.5|                 70|               US|      10 Years|
|  Plymouth Satellite|            18.0|              8|          318.0|        150.0|     3436|           11.0|                 70|               US|      10 Years|
|       AM

In [11]:
renamed_df = df.toDF("Car_New","MPG_New","Cylinders_New","Displacement_New","Horsepower_New","Weight_New","Acceleration_New","Model_New","Origin_New", "Warranty_New")
renamed_df.show()

+--------------------+-------+-------------+----------------+--------------+----------+----------------+---------+----------+------------+
|             Car_New|MPG_New|Cylinders_New|Displacement_New|Horsepower_New|Weight_New|Acceleration_New|Model_New|Origin_New|Warranty_New|
+--------------------+-------+-------------+----------------+--------------+----------+----------------+---------+----------+------------+
|Chevrolet Chevell...|   18.0|            8|           307.0|         130.0|      3504|            12.0|       70|        US|    10 Years|
|   Buick Skylark 320|   15.0|            8|           350.0|         165.0|      3693|            11.5|       70|        US|    10 Years|
|  Plymouth Satellite|   18.0|            8|           318.0|         150.0|      3436|            11.0|       70|        US|    10 Years|
|       AMC Rebel SST|   16.0|            8|           304.0|         150.0|      3433|            12.0|       70|        US|    10 Years|
|         Ford Torino|   17